In [1]:
# Importing the packages we will need
from __future__ import print_function
from __future__ import division
from IPython.core.display import HTML
from IPython.display import display, clear_output, FileLink
from ipywidgets import Button, Layout, GridBox, ButtonStyle, HBox, VBox, Label, Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import ipywidgets as ipw
%matplotlib inline
import pandas as pd
import math
import numpy as np
import xlrd
import xlsxwriter
import os.path
import glob
import warnings
import platform

HTML("""
<style>
#notebook-container{
    box-shadow: none !important;
}

.body {
    max-width: 960px !important;
}

.container {
    max-width: 960px !important;
}

.notebook_app {
    background: #fff !important;
}

.navbar-default {
    background: none;
    border: none;
}

.navbar-default .navbar-nav > li > a:hover, #kernel_indicator:hover {
    border-bottom: 2px solid #fff;
    color: rgba(255, 255, 255, 1);
}

div.input_area {
    border: none;
    border-radius: 0;
    background: #f7f7f7;
    line-height: 1.5em;
    margin: 0.5em 0;
    padding: 0;
    max-width: 960px !important;
}

div.cell {
    transition: all 0.25s;
    border: none;
    position: relative;
    top: 0;
    max-width: 960px !important;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border: none;
    background: transparent;
    box-shadow: 0 6px 18px #aaa;
    z-index: 10;
    top: -10px;
    max-width: 960px !important;
}


div#pager {
    opacity: 0.85;
    z-index: 9999;
    max-width: 960px !important;
}

.navbar-default .navbar-nav > .open > a, .navbar-default .navbar-nav > .open > a:hover, .navbar-default .navbar-nav > .open > a:focus {
    color: #fff;
    background-color: transparent;
    border-bottom: 2px solid #fff;
}

.dropdown-menu {
    z-index: 999999 !important;
    opacity: 0.95;
    max-width: 960px !important;
}

.dropdown-menu > li > a {
    color: #fff;
}

.dropdown-menu > .disabled > a, .dropdown-menu > .disabled > a:hover, .dropdown-menu > .disabled > a:focus {
    color: rgba(255, 255, 255, 0.25);
}

.navbar-nav > li > .dropdown-menu {
    border: none;
    box-shadow: none;
}

div.output_wrapper {
    background: #fff;
}

div.cell.selected .div.output_scroll {
    box-shadow: none;
    max-width: 960px !important;
}

div.output_wrapper {
    margin: 0 0 1em;
    transition: all 0.25s;
    max-width: 960px !important;
}

div.cell.selected .output_wrapper {
    margin: 0;
}

.dataframe {
    background: #fff;
    box-shadow: 0px 1px 2px #bbb;
}

.dataframe thead th, .dataframe tbody td {
    text-align: right;
    padding: 1em;
}

.output, div.output_scroll {
    box-shadow: none;
}

.rendered_html pre code {
    background: #f4f4f4;
    border: 1px solid #ddd;
    border-left: 3px solid #2a7bbd;
    color: #444;
    page-break-inside: avoid;
    font-family: monospace;
    font-size: 15px;
    line-height: 1.6;
    margin-bottom: 1.6em;
    max-width: 100%;
    overflow: auto;
    padding: 1em 1.5em;
    display: block;
    word-wrap: break-word;
}

h1, .h1 {
    font-size: 33px;
    font-family: "Trebuchet MS";
    font-size: 2.5em !important;
    color: #2a7bbd;
}
</style>
""")

In [3]:
print('Python is running, Python version is ')
print(platform.python_version())
print('App version 0.10.2, author Marianne Haseloff')

Python is running, Python version is 
3.7.4
App version 0.10.2


<div class="body">
This practical uses a full energy balance model to estimate glacier surface melt. The model is fully described by Brock and Arnold (2000); electronic copies of Earth Surface Processes and Landforms are available online (<a href="https://doi.org/10.1002/1096-9837(200006)25:6<649::AID-ESP97>3.0.CO;2-U">link to paper</a>).

When using this model it is useful to know the following:

<ul>
<li> Aspect of a slope is measured as degrees away from due North (i.e., <b>a North-facing glacier has aspect 0 degrees, and a South-facing glacier has aspect 180 degrees</b>) </li>
<li> All fluxes are considered positive when directed towards the surface </li>
<li> Fluxes are evaluated in SI units (W m$^{-2}$), BUT hourly totals of individual fluxes (SW, LW, sensible heat and latent heat) are outputted in mm of water equivalent (mm w.e.) </li>
<li> Consequently individual hourly fluxes may be negative, BUT negative fluxes in mm w.e. do not mean an increase in glacier ice, rather this just an intermediary calculation towards total melt (MLT) </li>
<li> Solheimajokull is located at 63.5 N and 19.5 W</li>
<li> On 26-27 June sunrise is ~3 am and sunset is ~11.30 pm</li>
</ul>

The model is completely contained in the web app. To run it, enter suitable values for <B>albedo, aspect, roughness, and slope</B> of the glacier below, and click on the button "Run the model". A link to a downloadable .xlsx spreadsheet will appear when the model is done, clicking on this link will download the model. The spreadsheet contains both the model inputs and the model outputs.</div>

<h3> Model inputs consist of: </h3>
<ol><li><h4> a list of site details (provided by the user):</h4>
    <ul>
    <li> <B>Latitude (deg)</B> (cell A13 of the output file) </li>
    <li> <B>Longitude (deg)</B> (cell A15 of the output file)</li>
    <li> <B>Slope (deg)</B> (cell A17 of the output file)</li>
    <li> <B>Aspect (deg)</B> (cell A19 of the output file)</li>
    <li> <B>Elevation (m)</B> (cell A21 of the output file)</li>
    <li> <B>Albedo</B> (cell A23 of the output file) </li>
    <li> <B>Roughness (m)</B> (cell A25 of the output file)</li>
    <li> <B>Lapse rate (degrees/m)</B> (cell A29 of the output file)</li>
    </ul></li>
    <li><h4> a timeseries of observations (provided with the model)</h4>
    <ul>
    <li> <B>Day</B> (column F of the output file)</li>
    <li> <B>Time</B> (column G of the output file)</li>
    <li> <B>Decimal Time</B> (column H of the output file)</li>
    <li> <B>Incoming SW (W/m$^2$) </B> (column I of the output file)</li>
    <li> <B>Vapour pressure (Pa)</B>  (column J of the output file)</li>
    <li> <B>Air temperature (°C)</B>  (column K of the output file)</li>
    <li> <B>Wind Speed (m/s)</B>  (column L of the output file)</li>
    </ul></li>
</ol>

In [4]:
# Defining a function plot that plots y against x and creates labels
def plot(x, y, ax, title, y_label):
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.plot(x, y)
    ax.margins(x=0, y=0)
    
def read_data():
    
    # Reading the meteorological data file 
    # df = pd.read_excel('/ml_blob_storage/Samplmet.xls')
    
    # Load data
    try:
        # You need to use "/ml_blob_storage" as a mount path to mount Azure Storage
        # in Azure Web App configuration
        df = pd.read_excel('https://www.dropbox.com/s/p6stvf969iuee5f/Samplmet.xls?dl=1')  # This path is used to load data for web app
    except FileNotFoundError:
        # local (on Azure Machine Learning) data file structure:
        # process_data.ipynb
        # |-- data /
        # |   |-- data_for_plots.pkl
        # |-- dashbaord /
        # |   |-- dashboard.ipynb
        # |   |-- Dockerfile
        # |   |-- requirements.txt
        df = pd.read_excel("./Samplmet.xls")  # This path is used to load data while working in AMLS environment

    # Renaming the columns 
    df.columns = ['day', 'time', 'inswrad', 'avp', 'airtemp','windspd']

    df['day'] = df['day'].astype(float)
    df['time'] = df['time'].astype(float)
    df['inswrad'] = df['inswrad'].astype(float)
    df['avp'] = df['avp'].astype(float)
    df['airtemp'] = df['airtemp'].astype(float)
    df['windspd'] = df['windspd'].astype(float)
    
    return df

In [5]:
def calcule_swr(slope,aspect,albedo,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,df):

    time_array = np.asarray(df.time)
    day_array  = np.asarray(df.day)
    inswrad_array = np.asarray(df.inswrad)

    dayangle= (day_array/365.25)*360
    eq_time = -0.128*np.sin( (dayangle-2.8)*math.pi/180 ) - 0.165*np.sin( (2*dayangle+19.7) * math.pi/180 )
    solhour = time_array/100 + (long - ref_long)/15 + eq_time - summertime
    c_1     = -23.2559 * np.cos( 2*math.pi*day_array/365 + 0.1582 )
    c_2     = -0.3915  * np.cos( 4*math.pi*day_array/365 + 0.0934 )
    c_3     = -0.1764  * np.cos( 6*math.pi*day_array/365 + 0.4539 )
    soldec  = 0.3948 + c_1 + c_2 + c_3
    solhran = 15 * ( solhour-12 )
    solaltr = np.arcsin( np.sin(lat*math.pi/180) * np.sin(soldec*math.pi/180) + np.cos(lat*math.pi/180) * np.cos(soldec*math.pi/180) * np.cos(solhran*math.pi/180) )
    solaltd = solaltr*180/math.pi
    cossolaz= np.sin(lat*math.pi/180) * np.sin(solaltr) - np.sin(soldec*math.pi/180) / ( np.cos(lat*math.pi/180)*np.cos(solaltr) )
    sinsolaz= np.cos(soldec*math.pi/180) * np.sin(solhran*math.pi/180) / np.cos(solaltr)
    solaz   = np.array([-np.arccos(cossolaz[i]) if value<0 else np.arccos(cossolaz[i]) for i,value in enumerate(sinsolaz)])
    solaz360= np.array(solaz) * 180/math.pi + 180
    cloudn  = 1-(inswrad_array/( 1368 * 0.75**(100000/(100000-elevation*10)) * np.cos(1.57-solaltr) )) #not sure this is correct
    cloudn  = np.array([1 if x>1 else x for x in cloudn])
    cloudn  = np.array([0 if x<0 else x for x in cloudn])
    diffuser= np.array([0.8 if x>0.8 else 0.65*x+0.15 for x in cloudn])
    directr = 1 - diffuser
    Qnormal = directr * inswrad_array/np.sin(solaltr)
    Qin     = Qnormal * ( np.sin(solaltr) * np.cos(slope*math.pi/180) + np.cos(solaltr) * np.sin(slope*math.pi/180) * np.cos(solaz-aspect*math.pi/180) )
    Q_SWR_SI= (1-albedo)*Qin +  (1-albedo)*diffuser*( inswrad_array * np.cos(slope*math.pi/180/2)**2 + albedo*np.sin(slope*math.pi/180/2)**2 )
    swr_array = Q_SWR_SI*0.0107784
    swr_array = np.array([0 if x<0 else x for x in swr_array])

    return swr_array, cloudn

In [6]:
def calcule_lwr(slope,aspect,albedo,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,df,cloudn):

    time_array = np.asarray(df.time)
    day_array  = np.asarray(df.day)
    inswrad_array = np.asarray(df.inswrad)
    airtemp_array = np.asarray(df.airtemp)
    
    eo     = 8.733 * 0.001* ( airtemp_array - lapse_rate*(elevation-met_stat_elevation) + 273.16)**0.788   
    e_star = ( 1 + (0.26* np.array([0 if x<0 else x for x in cloudn]) ) ) * eo
    lwin   = e_star * (5.7*(10**(-8)))*(((airtemp_array-(lapse_rate*(elevation-met_stat_elevation)))+273.16)**4)
    lwout  = 316
    lw_SI  = lwin-lwout
        
    lwr_array = lw_SI*0.0107784   
        
    return lwr_array

In [7]:
def calculat_turb_flx(slope,aspect,roughness,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,df,cloudn):
    warnings.filterwarnings("ignore")
    
    atmpress=100000*(1-elevation*0.0001)
    
    time_array = np.asarray(df.time)
    day_array  = np.asarray(df.day)
    avp_array  = np.asarray(df.avp)
    inswrad_array = np.asarray(df.inswrad)
    airtemp_array = np.asarray(df.airtemp)
    windspd_array = np.asarray(df.windspd)
    
    de = avp_array - 45*( elevation-met_stat_elevation)*lapse_rate - 610.8
    spehum = avp_array/atmpress
    spehtair = 1004.67 * ( 1 + 0.84*spehum )
    
    z_L = 0.2 * airtemp_array
    Qs_old = spehtair
    
    diff=10
    k=0
    
    while (diff>1e-3 and k<100):    
        u_star  =(0.4*windspd_array)/((np.log(2/roughness)) + 5*z_L )
        Re_star = u_star*roughness /0.00001461
        ln_zt   = np.log(roughness) + 0.317 - 0.565*np.log(Re_star) - 0.183*((np.log(Re_star))**2)
        ln_ze   = np.log(roughness)+0.396-(0.512*np.log(Re_star))-(0.18 *((np.log(Re_star))**2))

        Qs_SI   = (1.225*spehtair*0.16*windspd_array*(airtemp_array-(lapse_rate*(elevation-met_stat_elevation))))/(((np.log(2/roughness))+(5*(z_L)))*(((np.log(2))-ln_zt)+(5*(z_L))))
        Ql_SI   = (1.225*0.622*2500000*0.16*windspd_array*(de/atmpress))/(((np.log(2/roughness))+(5*(z_L)))*(((np.log(2))-ln_ze)+(5*(z_L))))
        
        Qs_SI = np.array([0 if value<1.0 else Qs_SI[i] for i,value in enumerate(windspd_array)])
        Qs_SI = np.array([0 if value<0.3 else Qs_SI[i] for i,value in enumerate(windspd_array/airtemp_array)])
        
        Ql_SI = np.array([0 if value<1.0 else Ql_SI[i] for i,value in enumerate(windspd_array)])
        Ql_SI = np.array([0 if value<0.3 else Ql_SI[i] for i,value in enumerate(windspd_array/airtemp_array)])
        
        
        Qs_SI = np.array([0 if math.isnan(x) else x for x in Qs_SI])
        Ql_SI = np.array([0 if math.isnan(x) else x for x in Ql_SI])
        
        Qs_SI = np.array([0 if x<0 else x for x in Qs_SI])
        Ql_SI = np.array([0 if x<0 else x for x in Ql_SI])
        
        L       = (1.225*(u_star**3)*((((airtemp_array-(lapse_rate*(elevation-met_stat_elevation)))+273.16)+273.16)/2)*spehtair)/(0.4*9.81*Qs_SI) 
        z_L     = 2/L
        
        diff = np.nansum(np.abs(Qs_old-Qs_SI))        
        Qs_old = Qs_SI
    
        k=k+1
    
    shf_array = Qs_SI*0.0107784
    lhf_array = Ql_SI*0.0107784
        
    return shf_array, lhf_array

In [8]:
def write_data(output_file,slope,aspect,albedo,roughness,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,df,swr_array,lwr_array,shf_array,lhf_array):
    
    time_array = np.asarray(df.time)
    day_array  = np.asarray(df.day)
    inswrad_array = np.asarray(df.inswrad)
    avp_array = np.asarray(df.avp)
    airtemp_array = np.asarray(df.airtemp)
    windspd_array = np.asarray(df.windspd)
    
    total_melt = swr_array + lwr_array + shf_array + lhf_array
    total_melt = np.array([0 if x<0 else x for x in total_melt])

    # Create a workbook and add a worksheet.
    
    out_str = ''.join(map(str, output_file))
    fileList = glob.glob("./output*xlsx")
    for filePath in fileList:
        os.remove(filePath)
        
    workbook = xlsxwriter.Workbook(out_str)
    worksheet = workbook.add_worksheet()

    # Write the site details used for creating the data   
    worksheet.write(10, 0, 'Site details')
    worksheet.write(11, 0, 'Latitude (deg)')
    worksheet.write(12, 0, lat)
    worksheet.write(13, 0, 'Longitude (deg)')
    worksheet.write(14, 0, long)
    worksheet.write(15, 0, 'Slope (deg)')
    worksheet.write(16, 0, slope)
    worksheet.write(17, 0, 'Aspect (deg)')
    worksheet.write(18, 0, aspect)
    worksheet.write(19, 0, 'Elevation (m)')
    worksheet.write(20, 0, elevation)
    worksheet.write(21, 0, 'Albedo ')
    worksheet.write(22, 0, albedo)
    worksheet.write(23, 0, 'Roughness (m)')
    worksheet.write(24, 0, roughness)
    worksheet.write(25, 0, 'Met. St. Elevation (m)')
    worksheet.write(26, 0, met_stat_elevation)
    worksheet.write(27, 0, 'Lapse Rate (deg/m) ')
    worksheet.write(28, 0, lapse_rate)
    
    
    # Start from the first cell. Rows and columns are zero indexed.
    row = 0
    col = 5

    worksheet.write(row, col, 'Day')
    worksheet.write(row, col+1, 'Time')
    worksheet.write(row, col+2, 'Decimal Time')
    worksheet.write(row, col+3, 'Incoming SW (W/m^2)')
    worksheet.write(row, col+4, 'Vapour pressure (Pa)')
    worksheet.write(row, col+5, 'Air temperature (°C)')
    worksheet.write(row, col+6, 'Wind Speed (m/s)')
    worksheet.write(row, col+7, 'SWR in mm w.e.')
    worksheet.write(row, col+8, 'LWR in mm w.e.')
    worksheet.write(row, col+9, 'SHF in mm w.e.')
    worksheet.write(row, col+10,'LHF in mm w.e.')
    worksheet.write(row, col+11,'MLT in mm w.e.')

    # Iterate over the data and write it out row by row.
    for x in swr_array:
        worksheet.write(row+1, col, day_array[row]) # first column is the day
        worksheet.write(row+1, col+1, time_array[row]) # first column is the time
        worksheet.write(row+1, col+2, day_array[row]+time_array[row]/2400) # first column is the time
        worksheet.write(row+1, col+3, inswrad_array[row]) # first column is the time
        worksheet.write(row+1, col+4, avp_array[row]) # first column is the time
        worksheet.write(row+1, col+5, airtemp_array[row]) # first column is the time
        worksheet.write(row+1, col+6, windspd_array[row]) # first column is the time
        worksheet.write(row+1, col+7, swr_array[row]) # first column is the time
        worksheet.write(row+1, col+8, lwr_array[row]) # first column is the time
        worksheet.write(row+1, col+9, shf_array[row]) # first column is the time
        worksheet.write(row+1, col+10, lhf_array[row]) # first column is the time
        worksheet.write(row+1, col+11, total_melt[row]) # first column is the time
        row += 1

        
    # Write a totals using a formula.
    worksheet.write(0, 0, 'Energy Flux Totals in mm w.e.')
    worksheet.write(1, 0, 'SWR')
    worksheet.write(2, 0, np.sum(swr_array))
    
    worksheet.write(1, 1, 'LWR')
    worksheet.write(2, 1, np.sum(lwr_array))
    
    
    worksheet.write(1, 2, 'SHF')
    worksheet.write(2, 2, np.sum(shf_array) )
    
    
    worksheet.write(1, 3, 'LHF')
    worksheet.write(2, 3, np.sum(lhf_array))
    
    
    worksheet.write(1, 4, 'MLT')
    worksheet.write(2, 4, np.sum(total_melt))

    
    # Write a totals using a formula.
    worksheet.write(4, 0, 'Energy Flux Rates in mm w.e. per day')
    worksheet.write(5, 0, 'SWR')
    worksheet.write(6, 0, np.sum(swr_array)/2)
    
    worksheet.write(5, 1, 'LWR')
    worksheet.write(6, 1, np.sum(lwr_array)/2)
    
    worksheet.write(5, 2, 'SHF')
    worksheet.write(6, 2, np.sum(shf_array)/2 )

    worksheet.write(5, 3, 'LHF')
    worksheet.write(6, 3, np.sum(lhf_array)/2)
    
    worksheet.write(5, 4, 'MLT')
    worksheet.write(6, 4, np.sum(total_melt)/2)
    
    workbook.close()

<div style="body"><h3>Model outputs are:</h3>

<ul>
<li> <B>Shortwave radiation in mm w.e.</B>  (column M)</li>
<li> <B>Longwave radiation in mm w.e.</B>  (column N)</li>
<li> <B>Sensible heat flux in mm w.e.</B>  (column O)</li>
<li> <B>Latent heat flux in mm w.e.</B>  (column P)</li>
<li> <B>Total melt in mm w.e.</B>  (column Q)</li>
</ul>

In addition, totals of these quantities over the data period (A3-E3) and per day (A7-E7) are calculated and provided in the output file as well. 

<h3>Run the model:</h3></div>

In [9]:
def getvalue(x): 
    return x

# Make user interface

style = {'description_width': '170px'}
w1 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=3.0,
    min=0,
    max=90.0,
    step=0.1,
    description='Slope in degrees:',
    style=style,
    disabled=False
))
#display(w1)

w2 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=90,
    min=0,
    max=360.0,
    step=0.1,
    description='Aspect in degrees:',
    style=style,
    disabled=False
))
#display(w2)


w3 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.31,
    min=0,
    max=1.0,
    step=0.01,
    description='Albedo:',
    style=style,
    disabled=False
))
#display(w3)

w4 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.005,
    min=0,
    max=1.0,
    step=0.0001,
    description='Roughness in m:',
    style=style,
    disabled=False
))
#display(w4)

w5 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=63.5,
    min=0,
    max=90.0,
    step=0.1,
    description='Latitude in degrees:',
    style=style,
    disabled=False
))
#display(w5)

w6 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=-19.5,
    min=-90,
    max=90.0,
    step=0.1,
    description='Longitude in degrees:',
    style=style,
    disabled=False
))
#display(w6)

w7 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=300.0,
    min=0,
    max=5000.0,
    step=0.1,
    description='Elevation in m:',
    style=style,
    disabled=False
))
#display(w7)

w8 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.0065,
    min=0,
    max=1.0,
    step=0.0001,
    description='Lapse rate in deg/m:',
    style=style,
    disabled=False
))
#display(w8)

In [10]:
left_box  = ipw.VBox([w1, w2, w3, w4])
right_box = ipw.VBox([w5, w6, w7, w8])
ipw.HBox([left_box,right_box])


In [11]:
# Loading the input parameters defined by the user
button = ipw.Button(description="Run the model!")
output = ipw.Output()

display(button, output)

global iter
iter=0

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Loading values and running the model.")
        slope=w1.result
        aspect=w2.result
        albedo=w3.result
        roughness=w4.result
        
        lat=w5.result
        long=w6.result
        elevation=w7.result
        lapse_rate=w8.result
    
        ref_long=0.0
        summertime=0.0
        met_stat_elevation=300.0
        
        global iter
        iter=iter+1 

        output_file=list('./output0.xlsx')
        output_file[8]=str(iter)
        
        print('The slope is %.1f, the aspect is %.1f, the albedo is %.4f, and the roughness is %.4f.' % (slope,aspect,albedo,roughness))
        
        data_in=read_data()
        [swr_array,cloudn] = calcule_swr(slope,aspect,albedo,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,data_in)
        lwr_array = calcule_lwr(slope,aspect,albedo,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,data_in,cloudn)
        [shf_array, lhf_array]=calculat_turb_flx(slope,aspect,roughness,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,data_in,cloudn)
        
        write_data(output_file,slope,aspect,albedo,roughness,lat,long,elevation,lapse_rate,ref_long,summertime,met_stat_elevation,data_in,swr_array,lwr_array,shf_array, lhf_array)
        #display(running_time)
        
        local_file = FileLink(''.join(map(str, output_file)), result_html_prefix="Click here to download: ")
        display(local_file)
        
button.on_click(on_button_clicked)

Button(description='Run the model!', style=ButtonStyle())

Output()